## Introduction
-----

In this assignment you will be recursively estimating a vehicle trajectory using available measurements and motion model. 

The vehicle is equipped with a LIDAR sensor, which returns range and bearing measurements corresponding to individual landmarks in the environment. The correspondences between landmarks and their global positions are assumed to be known beforehand. We also assume knowledge about which measurement belongs to which landmark.

## Motion and measurement models
-----

The motion model recieves odometry readings as inputs, and outputs the state (2d pose) of the vehicle $\mathbf{x}_k = \left[ x \, y \, \theta \right]^T$:

\begin{align}
\mathbf{x}_{k} &= \mathbf{x}_{k-1} + T
\begin{bmatrix}
\cos\theta_{k-1} &0 \\
\sin\theta_{k-1} &0 \\
0 &1
\end{bmatrix}
\left(
\begin{bmatrix}
v_k \\
\omega_k
\end{bmatrix}
+ \mathbf{w}_k
\right)
\, , \, \, \, \, \, \mathbf{w}_k = \mathcal{N}\left(\mathbf{0}, \mathbf{Q}\right)
\end{align}

The measurement model relates the current pose of the vehicle to the range and bearing measurements $\mathbf{y}^l_k = \left[r \, \phi \right]^T$.

\begin{align}
\mathbf{y}^l_k =
\begin{bmatrix}
\sqrt{(x_l - x_k - d\cos\theta_{k})^2 + (y_l - y_k - d\sin\theta_{k})^2} \\
atan2\left(y_l - y_k - d\sin\theta_{k},x_l - x_k - d\cos\theta_{k}\right) - \theta_k
\end{bmatrix}
+
\mathbf{n}^l_k
\, , \, \, \, \, \, \mathbf{n}^l_k = \mathcal{N}\left(\mathbf{0}, \mathbf{R}\right)
\end{align}

## Getting started
-----

Since the above models are nonlinear, we recommend using the Extended Kalman Filter (EKF) as the state estimator.
Specifically, you will need to provide code implementing the following steps:
- the prediction step, which uses odometry measurements and above motion model to provide a state and covariance estimate at a given timestep
- the correction step, which uses the range and bearing measurements provided by the LIDAR to correct the estimates

First, let's unpack the available data:

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

with open('data/data.pickle', 'rb') as f:
    data = pickle.load(f)

t = data['t']  # timestamps [s]

x_true = data['x_true']  # ground truth x position [m]
y_true = data['y_true']  # ground truth y position [m]
th_true = data['th_true']  # ground truth orientation [rad]

# input signal
v = data['v']  # translational velocity input [m/s]
om = data['om']  # rotational velocity input [rad/s]

# bearing and range measurements, LIDAR constants
b = data['b']  # bearing to each landmarks center in the frame attached to the laser [rad]
r = data['r']  # range measurements [m]
l = data['l']  # x,y positions of landmarks [m]
d = data['d']  # distance between robot center and laser rangefinder [m]
print(d)

Since the ground truth position and orientation are available, it is useful to plot them out before starting the assignment:

In [ ]:
gt_fig = plt.figure()
ax = gt_fig.add_subplot(111)
ax.plot(x_true, y_true)
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title('Ground Truth trajectory')
plt.show()

Now that our data is loaded, we can start getting things ready for our solver. One of the
most important aspects of a filter is setting the input andmeasurement noise covariance matrices, as well as the initial state and covariance values. We set the values here:

In [ ]:
v_var = 0.01  # translation velocity variance
om_var = 0.01  # rotational velocity variance
r_var = 0.1  # range measurements variance
b_var = 0.1  # bearing measurement variance

Q = np.diag([v_var, om_var]) # input noise covariance
R = np.diag([r_var, b_var])  # measurement noise covariance

x_est = np.zeros([len(v), 3])  # estimated states, x, y, and theta
P_est = np.zeros([len(v), 3, 3])  # state covariance matrices

x_est[0] = np.array([x_true[0], y_true[0], th_true[0]]) # initial state
P_est[0] = np.diag([1, 1, 0.1]) # initial state covariance

## Measurement update
-----
First, let's implement the measurement update function, which takes an available landmark measurement $l$ and updates the current state estimate $\mathbf{\check{x}}_k$.
For each landmark measurement in a given timestep $k$ you should implement the following steps:

\begin{align}
\mathbf{K}_k &= \mathbf{\check{P}}_k \mathbf{H}_k^T \left(\mathbf{H}_k \mathbf{\check{P}}_k \mathbf{H}_k^T + \mathbf{M}_k \mathbf{R}_k \mathbf{M}_k^T \right)^{-1} \\
\mathbf{\check{y}}^l_k &= \mathbf{h}_k\left(\mathbf{\check{x}}_k, \mathbf{0}\right) \\
\mathbf{\hat{x}}_k &= \mathbf{\check{x}} + \mathbf{K}_k \left(\mathbf{y}^l_k - \mathbf{\check{y}}^l_k\right)\\
\mathbf{\hat{P}}_k &= \left(\mathbf{1} - \mathbf{K}_k \mathbf{H}_k \right)\mathbf{\check{P}}_k
\end{align}

Assignment --> Lecture slide mapping:
Kk --> Kt
Pk --> Sigmat
Hk --> Gt
Mk --> Dt
Rk --> Qt
xk --> mut
hk() --> gt()

In [ ]:
from IPython.display import display
import sympy as sp
sp.init_printing()

def wraptopi(x):
    if x > np.pi:
        x = x - (np.floor(x / (2 * np.pi)) + 1) * 2 * np.pi
    elif x < -np.pi:
        x = x + (np.floor(x / (-2 * np.pi)) + 1) * 2 * np.pi
    return x

# TODO Implement this function, which should compute and return the measurement Jacobian
# H_k with respect to the state, x.
# params: xl - the x position of the input landmark.
#         yl - the y position of the input landmark.
#         x  - the current estimated state.
def measurement_jacobian(xl, yl, x):
    # Compute the partial derivatives of measurement model yk wrt xk
    # Where the formula for yk was given above

    # Define variables in sympy
    # Include d even though its value is always 0
    xl_sp, yl_sp, d_sp, theta_sp = sp.symbols('xl yl d theta')
    xk_sp, yk_sp = sp.symbols('xk yk')

    # Translate the given equations into sympy
    r = sp.sqrt((xl_sp - xk_sp - d_sp * sp.cos(theta_sp))**2 + (yl_sp - yk_sp - d_sp * sp.sin(theta_sp))**2)
    phi = sp.atan2(yl_sp - yk_sp - d_sp * sp.sin(theta_sp), xl_sp - xk_sp - d_sp * sp.cos(theta_sp)) - theta_sp

    # Use sympy to compute the measurement Jacobian
    H = sp.Matrix([r, phi]).jacobian(sp.Matrix([xk_sp, yk_sp, theta_sp]))

    # Substitute the function input into the Jacobian
    H = H.subs({xl_sp: xl, yl_sp: yl, xk_sp: x[0], yk_sp: x[1], theta_sp: x[2], d_sp: d[0]})

    # Convert the Jacobian into a numpy array for computations
    H = np.array(H).astype(np.float64)

    return H

# TODO Implement this function, which performs the Kalman update using the equations
# above.
# params: lk      - the array corresponding to the (x, y) coordinates of the landmark.
#         rk      - the current range measurement.
#         bk      - the current bearing measurement.
#         P_check - the current covariance estimate.
#         x_check - the current estimated state.
def measurement_update(lk, rk, bk, P_check, x_check):
    # 3.1 Compute measurement Jacobian using the landmarks and the current estimated state.
    H = measurement_jacobian(lk[0], lk[1], x_check)

    # 3.2 Compute the Kalman gain. 
    # Note: Mk = I (Dt in lecture notes), so Mk @ R @ Mk.T is the same as just R
    print(H.shape, P_check.shape, R.shape)
    K = P_check @ H.T @ np.linalg.inv(H @ P_check @ H.T + R)
    
    # 3.3 Correct the predicted state.
    # NB : Make sure to use wraptopi() when computing the bearing estimate!
    dx = lk[0] - x_check[0] - d[0] * np.cos(x_check[2])
    dy = lk[1] - x_check[1] - d[0] * np.sin(x_check[2])
    y_check = np.array([np.sqrt(dx**2 + dy**2), np.arctan2(dy, dx) - x_check[2]])
    y_check[1] = wraptopi(y_check[1])   # The predicted measurements

    y = np.array([rk, bk])              # The actual measurements

    x_check = x_check + K @ (y - y_check)
    x_check[2] = wraptopi(x_check[2])

    # 3.4 Correct the covariance.
    P_check = (np.eye(len(x_check)) - K @ H) @ P_check

    return x_check, P_check


## Main loop
-----
Now, implement the main filter loop, implementing the prediction step of the EKF using the provided motion model:

\begin{align}
\mathbf{\check{x}}_k &= \mathbf{f}\left(\mathbf{\hat{x}}_{k-1}, \mathbf{u}_{k-1}, \mathbf{0} \right) \\
\mathbf{\check{P}}_k &= \mathbf{F}_{k-1}\mathbf{\hat{P}}_{k-1}\mathbf{F}_{k-1}^T + \mathbf{L}_{k-1}\mathbf{Q}_{k-1}\mathbf{L}_{k-1}^T \, .
\end{align}

Where

\begin{align}
\mathbf{F}_{k-1} = \frac{\partial \mathbf{f}}{\partial \mathbf{x}_{k-1}}\bigg|_{\mathbf{\hat{x}}_{k-1},\mathbf{u}_{k},0}  \, , \, \, \, \,
\mathbf{L}_{k-1} = \frac{\partial \mathbf{f}}{\partial \mathbf{w}_{k}}\bigg|_{\mathbf{\hat{x}}_{k-1},\mathbf{u}_{k},0} \, .
\end{align}

In [ ]:
from tqdm import tqdm

# TODO Implement the main kalman filter loop, using the above equations.
# Make use of the measurement_update() function defined above.
#### 5. Main Filter Loop #######################################################################
for k in tqdm(range(1, len(t))):  # Start at 1 because we have initial prediction from ground truth.

    delta_t = t[k] - t[k - 1]  # time step (difference between timestamps)

    # 1. Update the state with IMU input.
    # NB : Make sure to use wraptopi() when computing the state heading!
    x_check = np.array([
        x_est[k-1, 0] + delta_t * v[k-1] * np.cos(x_est[k-1, 2]),
        x_est[k-1, 1] + delta_t * v[k-1] * np.sin(x_est[k-1, 2]),
        wraptopi(x_est[k-1, 2] + delta_t * om[k-1])
    ])

    # 1.1 Linearize Motion Model
    # Compute the Jacobian of f w.r.t. the last state.
    # Where xk = f(xk-1, ut, noise) was defined at the top
    # Directly derive the Jacobian because the variables are simple
    F = np.array([[1, 0, -delta_t * v[k-1] * np.sin(x_est[k-1, 2])],
                  [0, 1, delta_t * v[k-1] * np.cos(x_est[k-1, 2])],
                  [0, 0, 1]])

    # Compute the Jacobian w.r.t. the noise variables.
    # Directly derive the Jacobian because the variables are simple
    L = np.array([[delta_t * np.cos(x_est[k-1, 2]), 0],
                  [delta_t * np.sin(x_est[k-1, 2]), 0],
                  [0, delta_t]])
    
    # 2. Propagate uncertainty by updating the covariance.
    # Note: F and L are already based on the last time step and Q is constant, so only P_est has to be indexed
    P_check = F @ P_est[k-1] @ F.T + L @ Q @ L.T
    
    # 3. Update state estimate using available landmark measurements r[k], b[k].
    # Iterate through each landmark
    for i in range(len(r[k])):
        x_check, P_check = measurement_update(l[i], r[k][i], b[k][i], P_check, x_check)
        
    # Set final state predictions for this kth timestep.
    # Store the estimate and covariance for next iteration's calculations
    x_est[k] = x_check
    P_est[k] = P_check


Let's plot the resulting state estimates:

In [ ]:
gt_fig = plt.figure()
ax = gt_fig.add_subplot(111)
ax.plot(x_est[:, 0], x_est[:, 1])
ax.plot(x_true[:], y_true[:])
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title('Ground Truth trajectory')
plt.show()

x_fig = plt.figure()
ax = x_fig.add_subplot(111)
ax.plot(t, (x_est[:, 0]-x_true))
ax.set_xlabel('Time [s]')
ax.set_ylabel('x [m]')
ax.set_title('Ground Truth trajectory')

y_fig = plt.figure()
ax = y_fig.add_subplot(111)
ax.plot(t, (x_est[:, 1]-y_true))
ax.set_xlabel('Time [s]')
ax.set_ylabel('y [m]')
ax.set_title('Ground Truth trajectory')

th_fig = plt.figure()
ax = th_fig.add_subplot(111)
ax.plot(t, (x_est[:, 2]-th_true))
ax.set_xlabel('Time [s]')
ax.set_ylabel('theta [rad]')
ax.set_title('Ground Truth trajectory')
plt.show()

The plotted trajectory should match the ground truth. Congratulations! Assignment complete.